## TensorFlow, PyTorch Distributions

In [ ]:
import scipy.stats as ss
import numpy as np
import pandas as pd
import collections
import math
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.distributions import Bernoulli

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tf.contrib.distributions
# tfd = tfp.distributions
# tfb = tfp.distributions.bijectors

tfe = tf.contrib.eager
tfe.enable_eager_execution()
# sess = tf.InteractiveSession()
# sess.run(tf.global_variables_initializer())

from __future__ import print_function

print(tf.__version__)

##Helpful links for Distributions
# https://arxiv.org/pdf/1711.10604.pdf
# https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Understanding%20TensorFlow%20Distributions%20Shapes.ipynb
# https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html
# https://www.tensorflow.org/api_docs/python/tf/distributions/Normal

In [ ]:
##Tinker with tf distributions

dist = tf.distributions.Normal(loc=0., scale=3.)
print(dist.cdf(0.))
dist = tf.distributions.Normal(loc=[1, 2.], scale=[11, 22.])
# Evaluate the pdf of the first distribution on 0, and the second on 1.5, returning a length two tensor
print(dist.prob([0, 1.5]))

# Get 3 samples, returning a 3 x 2 tensor.
print(dist.sample([3]))
print(dist.sample([3]).numpy())

In [ ]:
##Tinker with pytorch distributions

from torch.distributions import Bernoulli

m = Bernoulli(torch.Tensor([0.3]))
r = []
for i in range(20):
    r.append(m.sample().numpy()[0])
print(r)  # 30% chance 1; 70% chance 0

In [ ]:
##Tinker with tf mixture of distributions
# https://www.tensorflow.org/api_docs/python/tf/contrib/distributions/Mixture

mixture_components = [1.0/2]*2
bimix_gauss = tfd.Mixture(
  cat=tfd.Categorical(probs=mixture_components),
  components=[
    tfd.Normal(loc=-1., scale=0.1),
    tfd.Normal(loc=+1., scale=0.5)
])

#Is Weibull bijector the only way to directly use weibull distribution in tf ?

x = tf.linspace(-2., 3., int(1e4))
bimix_gauss.prob(x)

## Scipy Distributions / Optimization

In [ ]:
print(np.random.binomial(10, 0.3, 10))
print(np.random.binomial(10, 0.3, 10).mean())

print(ss.binom(10, 0.3).rvs(10))
print(ss.binom(10, 0.3).cdf(5))
print(ss.binom(10, 0.3).mean())

In [ ]:
##Using scipy optimize with custom distributions
# https://www.projectrhea.org/rhea/index.php/MLE_Examples:_Exponential_and_Geometric_Distributions_Old_Kiwi

from scipy.stats import geom, gamma
from scipy.optimize import minimize

def neg_loglike(theta):
    prob = lambda x, y, z: -x * np.log(geom.pmf(y, z))
    data = [10, 9, 8, 7, 6, 5]
    return np.sum([prob(data[i], i+1, theta) for i in range(len(data))])

theta_start = 0.5
res = minimize(neg_loglike, theta_start, method = 'Nelder-Mead', options={'disp':True})
print(res)

print('------------')

def weibull_cdf(t, l, c):
    return 1 - math.exp(-l * (t**c))

def weibull_2seg_2clust_withspike_negloglike(theta): #TODO: Implement correct function later
    prob = lambda x, y, z: -x * np.log(gamma.pdf(y, z))
    data = [10, 9, 8, 7, 6, 5]
    return np.sum([prob(data[i], i+1, theta) for i in range(len(data))])

param_start = 0.5
res = minimize(weibull_2seg_2clust_withspike_negloglike, param_start, method = 'Nelder-Mead', options={'disp':True}) 
print(res)

In [ ]:
# https://github.com/scipy/scipy/issues/3056

import scipy.optimize as optimize

X = np.array([[1.020626, 1.013055], [0.989094, 1.059343]])
freq = 13.574380165289256
x_0 = [1., 1.]

def objective(b):
    def foo(r_log, freq):
        mu, sd = r_log.mean(), r_log.std()
        sd += 0.5 / freq
        return mu / sd * np.sqrt(freq)

    print(b)
    return -foo(np.log(np.maximum(np.dot(X - 1, b) + 1, 0.2)), freq=freq)

cons = ({'type': 'ineq', 'fun': lambda b: 2. - sum(b)},)
res = optimize.minimize(objective, x_0, bounds=[(0., 2.)]*len(x_0), constraints=cons, method='slsqp')
print(res)

## Generalized Linear Models

In [ ]:
##Poisson Regression
# https://stackoverflow.com/questions/47686227/poisson-regression-in-statsmodels-and-r

# import warnings
# warnings.filterwarnings('ignore')

import statsmodels.formula.api
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson

df = pd.DataFrame(np.random.randint(100, size=(50,2)))
df.rename(columns={0:'X1', 1:'X2'}, inplace=True)

# glm = statsmodels.formula.api.gee
# model = glm("X2 ~ X1", groups=None, data=df, family=Poisson())
# results = model.fit()
# print(results.summary())

# Add a column of ones for the intercept to create input X. This model is similar to R implementation
X = np.column_stack( (np.ones((df.shape[0], 1)), df.X1) )
y = df.X2
print(sm.GLM(y, X, family = Poisson()).fit().summary())

## Models related to Faders work 

In [ ]:
##Try implementing following papers using scipy, octave, tensorflow, pytorch
#https://repository.upenn.edu/cgi/viewcontent.cgi?referer=https://www.google.com/&httpsredir=1&article=1048&context=wharton_research_scholars
#https://pdfs.semanticscholar.org/d566/7b80c85b221aedf9c498fdd1d98a4478118b.pdf

In [ ]:
#Dummy data
ts_data1 = np.array([8,7,6,5,4,3,3,3,3,4,5,6,7,8,9,10])
ts_data2 = np.concatenate((5+ts_data1[0:8], ts_data1[8:]))
ts_data3 = np.concatenate((ts_data1[0:8], 5+ts_data1[8:]))
ts_data = np.array([ts_data1.cumsum(), ts_data2.cumsum(), ts_data3.cumsum()])
ts_data = np.tile(ts_data,(5,1))

# ts_data_context = np.array(['a','b','b','b','b','c','c','c','c','c','d','d','d','d','d','e'])
print(len(ts_data)) #,len(ts_data_context))

In [ ]:
num_clusters = 2
tot_num_clusters = 2 #num_clusters + 1
num_samples = len(ts_data)
num_customers_per_sample = list(map(lambda x : x.sum(), ts_data))

clust_assign = np.random.randint(num_clusters, size=num_samples)